In [1]:
%Addjar http://central.maven.org/maven2/org/twitter4j/twitter4j-core/4.0.4/twitter4j-core-4.0.4.jar

Starting download from http://central.maven.org/maven2/org/twitter4j/twitter4j-core/4.0.4/twitter4j-core-4.0.4.jar
Finished download of twitter4j-core-4.0.4.jar


In [2]:
%Addjar https://github.com/ibm-cds-labs/spark.samples/raw/master/dist/streaming-twitter-assembly-1.4.jar

Starting download from https://github.com/ibm-cds-labs/spark.samples/raw/master/dist/streaming-twitter-assembly-1.4.jar
Finished download of streaming-twitter-assembly-1.4.jar


In [3]:
import twitter4j.TwitterFactory
import twitter4j.Twitter
import twitter4j.conf.ConfigurationBuilder
import scala.collection.JavaConverters._
import twitter4j.Query

In [4]:
val cb = new ConfigurationBuilder()

cb.
setDebugEnabled(true).
setOAuthConsumerKey("xxxxx").
setOAuthConsumerSecret("xxxxxxxx").
setOAuthAccessToken("xxxxxxxx").
setOAuthAccessTokenSecret("xxxxxxxx")

val tf = new TwitterFactory(cb.build())
val client = tf.getInstance

In [8]:
val query = new Query("#Oscars2016")
query.setCount(500)
query.setSince("2015-02-09")

var searchResultCount = 0
var lowestTweetId = java.lang.Long.MAX_VALUE
var totaltweets = scala.collection.mutable.Buffer.empty[twitter4j.Status]

In [11]:
do {
    var queryResult = client.search(query)

    searchResultCount = queryResult.getTweets.size
       for (tweet <- queryResult.getTweets.asScala) {
         totaltweets = totaltweets ++ queryResult.getTweets.asScala

        if (tweet.getId < lowestTweetId) {
            lowestTweetId = tweet.getId
            query.setMaxId(lowestTweetId)
        }
    }

} while (searchResultCount != 0 && searchResultCount % 100 == 0)

In [31]:
import java.io._
import collection.JavaConversions._
val w = new BufferedWriter(new FileWriter("TweetsFeb14"))
totaltweets.foreach(twt => w.write(twt.getText))
w.close()

In [12]:
totaltweets.size

3218

In [13]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types._
import com.ibm.cds.spark.samples._
import org.apache.spark.sql.Row

In [14]:
import com.google.common.base.CharMatcher

In [15]:
 val sqlContext = new SQLContext(sc)
 var schemaTweets : StructType = null
  if ( schemaTweets == null ){
    val schemaString = "author date lang text lat:double long:double"
    schemaTweets =
      StructType(
        schemaString.split(" ").map(
          fieldName => {
            val ar = fieldName.split(":")
            StructField(
                ar.lift(0).get, 
                ar.lift(1).getOrElse("string") match{
                  case "int" => IntegerType
                  case "double" => DoubleType
                  case _ => StringType
                },
                true)
          }
        ).union( 
            ToneAnalyzer.sentimentFactors.map( f => StructField( f._1, DoubleType )).toArray[StructField]
        )
      )
  }

In [16]:
import com.ibm.cds.spark.samples.config.DemoConfig
import org.http4s.client.blaze.PooledHttp1Client
import org.apache.spark.sql.Row

In [17]:
val config1 = new DemoConfig
val keys = config1.getConfig("tweets.key").split(",");
 //Tone Analyzer service credential copied from section above
    config1.setConfig("watson.tone.url","https://gateway.watsonplatform.net/tone-analyzer-experimental/api")
    config1.setConfig("watson.tone.password","xxxxxx")
    config1.setConfig("watson.tone.username","xxxxxxx")


In [18]:
  val broadcastVar = sc.broadcast(config1.toImmutableMap)

In [19]:
  val tweets = totaltweets.filter { status => 
    Option(status.getUser).flatMap[String] { 
      u => Option(u.getLang) 
    }.getOrElse("").startsWith("en") && CharMatcher.ASCII.matchesAllOf(status.getText) && ( keys.isEmpty || keys.exists{status.getText.contains(_)})
  }

In [20]:
import scala.collection.mutable._

In [21]:
   val rowTweets = tweets.map(status=> {
    lazy val client = PooledHttp1Client()
    val sentiment = ToneAnalyzer.computeSentiment( client, status, broadcastVar )

    var colValues = Array[Any](
      status.getUser.getName, //author
      status.getCreatedAt.toString,   //date
      status.getUser.getLang,  //Lang
      status.getText,               //text
      Option(status.getGeoLocation).map{ _.getLatitude}.getOrElse(0.0),      //lat
      Option(status.getGeoLocation).map{_.getLongitude}.getOrElse(0.0)    //long
      //exception
    )

    var scoreMap : Map[String, Double] = Map()
    if ( sentiment != null ){
      for ( tone <- Option( sentiment.children ).getOrElse( Seq() ) ){
        for ( result <- Option( tone.children ).getOrElse( Seq() ) ){
          scoreMap.put( result.id, result.normalized_score )
        }
      }
    }

    colValues = colValues ++ ToneAnalyzer.sentimentFactors.map { f => (BigDecimal(scoreMap.get(f._2).getOrElse(0.0)).setScale(2, BigDecimal.RoundingMode.HALF_UP).toDouble) * 100.0  }
    //Return [Row, (sentiment, status)]
    (Row(colValues.toArray:_*),(sentiment, status))
  })

In [22]:
val tweetsrdd = rowTweets.grouped(10).map(s => sc.parallelize(s))

In [23]:
var firstelement = tweetsrdd.toList(0).map(t=>t._1)

In [ ]:
var workingRDD = sc.parallelize(firstelement.collect())

In [21]:
tweetsrdd.foreach{rdd => {
        try{                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
              workingRDD = sc.parallelize( rdd.map( t => t._1 ).collect()).union(workingRDD)  

        }catch{
            case e: Exception => e.printStackTrace()
        }
}}

In [22]:
if ( workingRDD.count <= 0 ){
      println("No data receive. Please start the Twitter stream again to collect data")
    }
    
    try{
      val df = sqlContext.createDataFrame( workingRDD, schemaTweets )
      df.registerTempTable("tweets")
      
      println("A new table named tweets with " + df.count() + " records has been correctly created and can be accessed through the SQLContext variable")
      println("Here's the schema for tweets")
      df.printSchema()
      
    }catch{
      case e: Exception => {e.printStackTrace();}
    }

A new table named tweets with 10 records has been correctly created and can be accessed through the SQLContext variable
Here's the schema for tweets
root
 |-- author: string (nullable = true)
 |-- date: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- text: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- Cheerfulness: double (nullable = true)
 |-- Negative: double (nullable = true)
 |-- Anger: double (nullable = true)
 |-- Analytical: double (nullable = true)
 |-- Confident: double (nullable = true)
 |-- Tentative: double (nullable = true)
 |-- Openness: double (nullable = true)
 |-- Agreeableness: double (nullable = true)
 |-- Conscientiousness: double (nullable = true)


In [23]:
val fullSet = sqlContext.sql("select * from tweets")

In [24]:
fullSet.show

+--------------------+--------------------+----+--------------------+---+----+------------+--------+-----+----------+---------+---------+-----------------+-------------+-----------------+
|              author|                date|lang|                text|lat|long|Cheerfulness|Negative|Anger|Analytical|Confident|Tentative|         Openness|Agreeableness|Conscientiousness|
+--------------------+--------------------+----+--------------------+---+----+------------+--------+-----+----------+---------+---------+-----------------+-------------+-----------------+
|          Oscar Pitt|Mon Feb 15 12:13:...|  en|RT @cinemasTV: #M...|0.0| 0.0|         0.0|     0.0|  0.0|       0.0|      0.0|      0.0|             97.0|          0.0|             68.0|
|          Oscar Pitt|Mon Feb 15 12:13:...|  en|RT @MadeInMilan_0...|0.0| 0.0|        97.0|     0.0|  0.0|       0.0|      0.0|     85.0|             48.0|         50.0|            100.0|
|          Oscar Pitt|Mon Feb 15 12:13:...|  en|RT @guaubc: 